In [1]:
import numpy as np
import healpy as hp

from polymv import polymv
from polymv import utils

Let's create a test set of $a_{\ell m}$s:

In [2]:
lmax = 1500

In [3]:
np.random.seed(1)
alm = hp.synalm(np.ones(2000), lmax=lmax)
alm = np.column_stack([alm.real, alm.imag])
alm

array([[ 1.62434536,  0.        ],
       [-0.61175641,  0.        ],
       [-0.52817175,  0.        ],
       ...,
       [ 0.46115612,  0.59244991],
       [ 0.35090859,  1.18446272],
       [-0.5622994 , -0.69337834]], shape=(1127251, 2))

# `polymv.mvs`

To obtain multipole vectors:

In [4]:
help(polymv.mvs)

Help on cython_function_or_method in module polymv.polymv:

mvs(input1, input2, LMAX)
    Get multipole vectors in spherical coordinates for a multipole given a map based on lmax.

    Parameters:
        input1 (float): Real part of multipole moments from a map (Healpy indexing).
        input2 (float): Imaginary part of multipole moments from a map (Healpy indexing).
        LMAX (int): Maximum l value to compute the multipole vectors.

    Returns:
        array: An array for all multipole vectors in spherical coordinates (theta, phi in radians) from l=2 to lmax.



Let's extract the multipole vectors from $\ell=2$ to $\ell=1500$:

In [5]:
%%time
mvs_1500 = polymv.mvs(alm[:, 0], alm[:, 1], lmax)
np.vstack(mvs_1500).shape, mvs_1500

CPU times: user 26min 12s, sys: 16.3 s, total: 26min 28s
Wall time: 7min 12s


((2, 2251498),
 (array([1.83441666, 1.37302603, 1.30717599, ..., 0.07286444, 0.04895423,
         0.01967822], shape=(2251498,)),
  array([ 0.12767786,  1.33828394, -3.01391479, ...,  2.06658143,
         -2.92918786,  0.48225845], shape=(2251498,))))

# `polymv.otherfuncs`

In [6]:
help(utils)

Help on module polymv.utils in polymv:

NAME
    polymv.utils

FUNCTIONS
    get_mvs_from_many(mvs, lin, lout, l)
        Get specific multiple vectors given a list of many (output from mvs).

        Args:
            mvs (np.ndarray): array containing multipole vectors [theta, phi].
            lin (int): initial multipole.
            lout (int): final multipole.
            l (int): multipole.

        Returns:
            np.ndarray.

    mvs_north(mvs)
        Get multipole vectors on north hemisphere.

        Args:
            mvs (np.ndarray): array containing multipole vectors [theta, phi].

        Returns:
            np.ndarray [theta, phi] in radians.

    to_cart(theta_phi_array)
        Convert spherical to Cartesian coordinate. This function works only for a single multipole or stacked arrays, e.g. 2 and 3 MVs together.

        Args:
            theta_phi_array (np.ndarray): array containing in radians [theta, phi].

        Returns:
            np.ndarray.

DATA
    

Extract specific multipole vectors from many others:

In [7]:
utils.get_mvs_from_many(np.vstack(mvs_1500).T, 2, lmax, 3)

array([[ 2.73836992,  2.08191869],
       [ 1.52087956,  0.19300013],
       [ 1.62071309, -2.94859252],
       [ 2.34022757,  0.13144151],
       [ 0.40322273, -1.05967396],
       [ 0.80136509, -3.01015115]])

In [8]:
utils.get_mvs_from_many(np.vstack(mvs_1500).T, 2, lmax, 5)

array([[ 1.88305141, -3.08268267],
       [ 2.0002518 , -1.95979302],
       [ 2.39314475, -0.68251023],
       [ 1.80531117,  0.80260215],
       [ 2.58851484,  1.73458048],
       [ 1.25854124,  0.05890998],
       [ 1.14134086,  1.18179963],
       [ 0.7484479 ,  2.45908243],
       [ 1.33628148, -2.33899051],
       [ 0.55307782, -1.40701218]])

Multipole vectors in Cartesian coordinates:

In [9]:
help(utils.to_cart)

Help on cython_function_or_method in module polymv.utils:

to_cart(theta_phi_array)
    Convert spherical to Cartesian coordinate. This function works only for a single multipole or stacked arrays, e.g. 2 and 3 MVs together.

    Args:
        theta_phi_array (np.ndarray): array containing in radians [theta, phi].

    Returns:
        np.ndarray.



In [10]:
utils.to_cart(np.vstack(mvs_1500).T)

array([[ 0.95759438,  0.12293233, -0.26057752],
       [ 0.22593142,  0.95412221,  0.19648358],
       [-0.95759438, -0.12293233,  0.26057752],
       ...,
       [-0.03463258,  0.06403453,  0.99734656],
       [-0.04783496, -0.01031598,  0.99880198],
       [ 0.01743279,  0.0091258 ,  0.99980639]], shape=(2251498, 3))

Obtain multipole vectors on north hemisphere:

In [11]:
help(utils.mvs_north)

Help on cython_function_or_method in module polymv.utils:

mvs_north(mvs)
    Get multipole vectors on north hemisphere.

    Args:
        mvs (np.ndarray): array containing multipole vectors [theta, phi].

    Returns:
        np.ndarray [theta, phi] in radians.



In [12]:
np.vstack(mvs_1500).T.shape

(2251498, 2)

In [13]:
utils.mvs_north(np.vstack(mvs_1500).T).shape, utils.mvs_north(np.vstack(mvs_1500).T)

((1125749, 2),
 array([[ 1.37302603,  1.33828394],
        [ 1.30717599, -3.01391479],
        [ 1.52087956,  0.19300013],
        ...,
        [ 0.07286444,  2.06658143],
        [ 0.04895423, -2.92918786],
        [ 0.01967822,  0.48225845]], shape=(1125749, 2)))

# `polymv.fvs`:

In [14]:
help(polymv.fvs)

Help on cython_function_or_method in module polymv.polymv:

fvs(input1, input2, LMAX)
    Get Fréchet mean vectors in spherical coordinates for a multipole given a map based on lmax.

    Parameters:
        input1 (float): Multipole vector theta coordinate.
        input2 (float): Multipole vector phi coordinate.
        LMAX (int): Maximum l value to compute the Fréchet mean.

    Returns:
        array: An array for all Fréchet mean based on all multipole vectors in spherical coordinates (theta, phi) from l=2 to lmax.



In [15]:
%%time
polymv.fvs(mvs_1500[0], mvs_1500[1], lmax)

CPU times: user 6min 5s, sys: 437 ms, total: 6min 6s
Wall time: 2min 35s


(array([0.39372199, 1.30678037, 1.41442982, ..., 0.79901509, 1.2620573 ,
        0.46823184], shape=(1498,)),
 array([5.54727987, 1.64990686, 0.53026658, ..., 5.88028155, 0.76763323,
        1.55541193], shape=(1498,)))